# Hosting Strands + LangGraph agent with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a LangGraph with Amazon Bedrock model example. For Strands Agents with Amazon Bedrock model check [here](../01-strands-with-bedrock-model)
and for a Strands Agents with an OpenAI model check [here](../03-strands-with-openai-model).

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                               |
| Agent type          | Single                                                                       |
| Agentic Framework   | Strands + LangGraph                                                                    |
| LLM model           | Anthropic Claude Sonnet 3                                                    |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands + LangGraph and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Easy                                                                         |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                 |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a LangGraph agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using LangGraph


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker running

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

In [15]:
from textwrap import dedent
from src.graph import build_graph
from src.utils.common_utils import get_message_from_string

In [16]:
%%writefile runtime.py

"""
Entry point script for the LangGraph Demo.
"""
import os
import sys
import json
import shutil
import asyncio
import argparse
from src.workflow import run_graph_streaming_workflow

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수

    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

async def graph_streaming_execution(payload):

    user_query = payload.get("prompt")

    """Execute full graph streaming workflow with real-time events"""
    remove_artifact_folder()

    print("\n=== Starting Graph Streaming Execution ===")
    print("Real-time streaming events from full graph:")

    try:
        async for event in run_graph_streaming_workflow(user_input=user_query, debug=False):
            #if "data" in event:
            yield event

    except Exception as e:
        # Handle errors gracefully in streaming context
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"Streaming error: {error_response}")
        yield error_response

    # async for event in run_graph_streaming_workflow(user_input=user_query, debug=False):



async def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()

    async for event in graph_streaming_execution(json.loads(args.payload)):

        #print (event)
        if event.get("event_type") == "text_chunk":
            # Print streaming text chunks in real-time
            print(event.get("data", ""), end="", flush=True)
            #print (event)
        elif event.get("event_type") == "reasoning":
            # Print reasoning tokens in real-time (can be used separately later)
            print(event.get("reasoning_text", ""), end="", flush=True)
            print(event.get("data", ""), end="", flush=True)
            #print (event)

        ## 툴 프린팅은 수정해야 함!! 스트리밍 될 수 있는 지 확인하기 event를 출력하면 어떻게 넘어 오는지 확인 가능함. 
        elif event.get("event_type") == "tool_use": 
            # Print tool usage events
            tool_name = event.get("tool_name", "unknown")
            tool_input = event.get("tool_input", "")
            #print (event)

            # Try to parse tool_input as JSON and extract task
            try:
                tool_data = json.loads(tool_input)
                task = tool_data.get("task", tool_input)
                #print(f"\n[TOOL] Using {tool_name}", flush=True)
                print(f"[TOOL] Task: {task}{'...' if len(task) > 100 else ''}", flush=True)
            except:
                pass
                #print(f"\n[TOOL] Using {tool_name}...", flush=True)
        elif event.get("type") == "final_result":
            print(f"\n\n[FINAL] Agent: {event.get('agent')}")
            print(f"[FINAL] Response: {event.get('response')}")
        else:
            # Print other events
            print(f"\n[EVENT] {event}")

if __name__ == "__main__":
    asyncio.run(main())


Overwriting runtime.py


#### Invoking local agent

In [17]:
!python runtime.py '{"prompt": "이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 './data/Dat-fresh-food-claude.csv' 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요. 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요."}'

'./artifacts/' 폴더가 존재하지 않습니다.

=== Starting Graph Streaming Execution ===
Real-time streaming events from full graph:

INFO [src.workflow] ===== Starting graph streaming workflow =====

INFO [src.workflow] 
user input: 이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 ./data/Dat-fresh-food-claude.csv 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요. 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.

INFO [src.graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled
handoff_to_planner: I'll need to consult our planning system for this request.

이 요청은 데이터 분석, 시각화, PDF 리포트 생성 등 복잡한 작업들을 포함하고 있어 플래너의 도움이 필요합니다. 플래너가 체계적인 분석 계획을 수립하고 단계별 실행을 도와드릴 수 있도록 하겠습니다.
INFO [src.graph.nodes] ===== Coordinator updated shared global state with 4 keys =====

INFO [src.graph.nodes] ===== Coordinator completed t

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agent SDK + LangGraph with Amazon Bedrock model
Let's start with our Strands Agent SDK + LangGraph using Amazon Bedrock model. Other examples with different frameworks and models are available in the parent directories

### Dockerfile


In [ ]:
%%writefile runtime.py

"""
Entry point script for the LangGraph Demo.
"""
import os
import sys
import json
import shutil
import asyncio
import argparse
from src.workflow import run_graph_streaming_workflow
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")


@app.entrypoint
async def graph_streaming_execution(payload):

    user_query = payload.get("prompt")

    """Execute full graph streaming workflow with real-time events"""
    remove_artifact_folder()
    
    print("\n=== Starting Graph Streaming Execution ===")
    print("Real-time streaming events from full graph:")
    
    async for event in run_graph_streaming_workflow(user_input=user_query, debug=False):
        if event.get("event_type") == "text_chunk":
            # Print streaming text chunks in real-time
            print(event.get("data", ""), end="", flush=True)
            #print (event)
        elif event.get("event_type") == "reasoning":
            # Print reasoning tokens in real-time (can be used separately later)
            print(event.get("reasoning_text", ""), end="", flush=True)
            #print (event)
        
        ## 툴 프린팅은 수정해야 함!! 스트리밍 될 수 있는 지 확인하기 event를 출력하면 어떻게 넘어 오는지 확인 가능함. 
        elif event.get("event_type") == "tool_use": 
            # Print tool usage events
            tool_name = event.get("tool_name", "unknown")
            tool_input = event.get("tool_input", "")
            #print (event)
            
            # Try to parse tool_input as JSON and extract task
            try:
                import json
                tool_data = json.loads(tool_input)
                task = tool_data.get("task", tool_input)
                #print(f"\n[TOOL] Using {tool_name}", flush=True)
                print(f"[TOOL] Task: {task}{'...' if len(task) > 100 else ''}", flush=True)
            except:
                pass
                #print(f"\n[TOOL] Using {tool_name}...", flush=True)
        elif event.get("type") == "final_result":
            print(f"\n\n[FINAL] Agent: {event.get('agent')}")
            print(f"[FINAL] Response: {event.get('response')}")
        else:
            # Print other events
            print(f"\n[EVENT] {event}")
    
    # Print the conversation history from global state
    print("\n\n=== Conversation History ===")
    from src.graph.nodes import _global_node_states
    shared_state = _global_node_states.get('shared', {})
    history = shared_state.get('history', [])
    
    if history:
        for hist_item in history:
            print("===")
            print(f'agent: {hist_item["agent"]}')
            print(f'message: {hist_item["message"]}')
    else:
        print("No conversation history found in global state")
    
    print("\n=== Graph Streaming Execution Complete ===")



if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = asyncio.run(graph_streaming_execution(json.loads(args.payload)))
    print(response)

Overwriting runtime.py


In [25]:
%%writefile runtime.py


"""
Entry point script for the LangGraph Demo.
"""
import os
import shutil
import argparse
from src.workflow import run_graph_streaming_workflow
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수

    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

@app.entrypoint
async def graph_streaming_execution(payload):

    print ("111111")
    user_query = payload.get("prompt")
    print ("222222")

    """Execute full graph streaming workflow with real-time events"""
    remove_artifact_folder()

    print("\n=== Starting Graph Streaming Execution ===")
    print("Real-time streaming events from full graph:")

    try:
        async for event in run_graph_streaming_workflow(user_input=user_query, debug=False):
            #if "data" in event:
            print (event)
            yield event
            
    except Exception as e:
        # Handle errors gracefully in streaming context
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"Streaming error: {error_response}")
        yield error_response

    # async for event in run_graph_streaming_workflow(user_input=user_query, debug=False):

if __name__ == "__main__":
    app.run()
    
    



Overwriting runtime.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating runtime role

Before starting, let's create an IAM role for our AgentCore Runtime. We will do so using the utils function pre-developed for you.

In [1]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from src.utils.agentcore import create_agentcore_role

agent_name="runtime"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

sys.path[0]: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications
Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': '66600022-ce77-418d-be32-1b8b959da237', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 27 Aug 2025 00:55:37 GMT', 'x-amzn-requestid': '66600022-ce77-418d-be32-1b8b959da237', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-runtime-role
recreating agentcore-runtime-role
attaching role policy agentcore-runtime-role


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

**Signature:**
```python
agentcore_runtime.configure(
    entrypoint: str,
    execution_role: Optional[str] = None,
    agent_name: Optional[str] = None,
    requirements: Optional[List[str]] = None,
    requirements_file: Optional[str] = None,
    ecr_repository: Optional[str] = None,
    container_runtime: Optional[str] = None,
    auto_create_ecr: bool = True,
    auto_create_execution_role: bool = False,
    authorizer_configuration: Optional[Dict[str, Any]] = None,
    region: Optional[str] = None,
    protocol: Optional[Literal['HTTP', 'MCP']] = None,
) -> bedrock_agentcore_starter_toolkit.operations.runtime.models.ConfigureResult
```

**Docstring:**  
Configure Bedrock AgentCore from notebook using an entrypoint file.

**Args:**
- **entrypoint**: Path to Python file with optional Bedrock AgentCore name (e.g., "handler.py" or "handler.py:bedrock_agentcore")
- **execution_role**: AWS IAM execution role ARN or name (optional if auto_create_execution_role=True)
- **agent_name**: name of the agent
- **requirements**: Optional list of requirements to generate requirements.txt
- **requirements_file**: Optional path to existing requirements file
- **ecr_repository**: Optional ECR repository URI
- **container_runtime**: Optional container runtime (docker/podman)
- **auto_create_ecr**: Whether to auto-create ECR repository
- **auto_create_execution_role**: Whether to auto-create execution role (makes execution_role optional)
- **authorizer_configuration**: JWT authorizer configuration dictionary
- **region**: AWS region for deployment
- **protocol**: agent server protocol, must be either HTTP or MCP

In [27]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    agent_name=agent_name,
    entrypoint="runtime.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/06_insight_extractor_strands_sdk_agentcore_stream/runtime.py, bedrock_agentcore_name=runtime
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint:Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/06_insight_extractor_strands_sdk_agentcore_stream/runtime.py, bedrock_agentcore_name=runtime
Configuring BedrockAgentCore agent: runtime
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Configuring BedrockAgentCore agent: runtime
Generated .dockerignore
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Generated .dockerignore
Generated Dockerfile: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/06_insight_extractor_strands_sdk_agentcore_stream/Dockerfile
INFO:bedrock_agentcore_starter_

ConfigureResult(config_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/06_insight_extractor_strands_sdk_agentcore_stream/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/06_insight_extractor_strands_sdk_agentcore_stream/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/06_insight_extractor_strands_sdk_agentcore_stream/.dockerignore'), runtime='Docker', region='us-west-2', account_id='615299776985', execution_role='arn:aws:iam::615299776985:role/agentcore-runtime-role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [28]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • No local Docker required
💡 Available deployment modes:
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentc

✅ Reusing existing ECR repository: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-runtime


✅ Execution role validation passed: arn:aws:iam::615299776985:role/agentcore-runtime-role
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:✅ Execution role validation passed: arn:aws:iam::615299776985:role/agentcore-runtime-role
Preparing CodeBuild project and uploading source...
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: runtime
INFO:bedrock_agentcore_starter_toolkit.services.codebuild:Getting or creating CodeBuild execution role for agent: runtime
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-d92c6a81b2
INFO:bedrock_agentcore_starter_toolkit.services.codebuild:Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-d92c6a81b2
Reusing existing CodeBuild execution role: arn:aws:iam::615299776985:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-d92c6a81b2
INFO:bedrock_agentcore_starter_toolkit.services.codebuild:Reusing existing Cod

In [29]:
import platform
print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"Architecture: {platform.architecture()}")
print(f"Platform: {platform.platform()}")

Machine: aarch64
Processor: aarch64
Architecture: ('64bit', 'ELF')
Platform: Linux-6.8.0-1033-aws-aarch64-with-glibc2.35


### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [30]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: runtime
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Retrieved Bedrock AgentCore status for: runtime


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [31]:
#invoke_response = agentcore_runtime.invoke({"prompt": "How much is 2+2?"})
invoke_response = agentcore_runtime.invoke({"prompt": '이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요. 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.'})
invoke_response

Invoking BedrockAgentCore agent 'runtime' via cloud endpoint
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.invoke:Invoking BedrockAgentCore agent 'runtime' via cloud endpoint


{"timestamp": "2025-08-26T14:14:19.671074", "session_id": "ABC", "agent_name": "coordinator", "source": 
"strands_data_analysis_graph", "type": "agent_text_stream", "event_type": "text_chunk", "data": "han", 
"chunk_size": 3}

{"timestamp": "2025-08-26T14:14:19.671510", "session_id": "ABC", "agent_name": "coordinator", "source": 
"strands_data_analysis_graph", "type": "agent_text_stream", "event_type": "text_chunk", "data": "doff_to_", 
"chunk_size": 8}

{"timestamp": "2025-08-26T14:14:19.672014", "session_id": "ABC", "agent_name": "coordinator", "source": 
"strands_data_analysis_graph", "type": "agent_text_stream", "event_type": "text_chunk", "data": "planner: I", 
"chunk_size": 10}

{"timestamp": "2025-08-26T14:14:19.672521", "session_id": "ABC", "agent_name": "coordinator", "source": 
"strands_data_analysis_graph", "type": "agent_text_stream", "event_type": "text_chunk", "data": "'ll need to cons",
"chunk_size": 16}

...

{"timestamp": "2025-08-26T14:14:45.352071", "session_id": "ABC", "agent_name": "supervisor", "source": 
"strands_data_analysis_graph", "type": "agent_tool_stream", "event_type": "tool_use", "tool_name": 
"coder_agent_tool", "tool_id": "tooluse_0DVfLJ8RQKO5PYmjw4KVpA", "tool_input": "{\"task\": \"We need to analyze 
Amazon product sales data. Please perform the following tasks:\\n\\n1. Create or download a sample Amazon product 
sales dataset suitable for analysis\\n2. Load the dataset\\n3. Examine basic properties (shape, info, 
describe)\\n4. Check for missing values and outliers\\n5. Perform basic statistical analysis (mean, median, 
variance)\\n6. Display a summary of your findings\\n\\nPlease provide code with explanations and output the initial
data exploration results in a clear, re"}

{"timestamp": "2025-08-26T14:14:45.352431", "session_id": "ABC", "agent_name": "supervisor", "source": 
"strands_data_analysis_graph", "type": "agent_tool_stream", "event_type": "tool_use", "tool_name": 
"coder_agent_tool", "tool_id": "tooluse_0DVfLJ8RQKO5PYmjw4KVpA", "tool_input": "{\"task\": \"We need to analyze 
Amazon product sales data. Please perform the following tasks:\\n\\n1. Create or download a sample Amazon product 
sales dataset suitable for analysis\\n2. Load the dataset\\n3. Examine basic properties (shape, info, 
describe)\\n4. Check for missing values and outliers\\n5. Perform basic statistical analysis (mean, median, 
variance)\\n6. Display a summary of your findings\\n\\nPlease provide code with explanations and output the initial
data exploration results in a clear, readable for"}

{"timestamp": "2025-08-26T14:14:45.352933", "session_id": "ABC", "agent_name": "supervisor", "source": 
"strands_data_analysis_graph", "type": "agent_tool_stream", "event_type": "tool_use", "tool_name": 
"coder_agent_tool", "tool_id": "tooluse_0DVfLJ8RQKO5PYmjw4KVpA", "tool_input": "{\"task\": \"We need to analyze 
Amazon product sales data. Please perform the following tasks:\\n\\n1. Create or download a sample Amazon product 
sales dataset suitable for analysis\\n2. Load the dataset\\n3. Examine basic properties (shape, info, 
describe)\\n4. Check for missing values and outliers\\n5. Perform basic statistical analysis (mean, median, 
variance)\\n6. Display a summary of your findings\\n\\nPlease provide code with explanations and output the initial
data exploration results in a clear, readable format"}

{"timestamp": "2025-08-26T14:14:45.353067", "session_id": "ABC", "agent_name": "supervisor", "source": 
"strands_data_analysis_graph", "type": "agent_tool_stream", "event_type": "tool_use", "tool_name": 
"coder_agent_tool", "tool_id": "tooluse_0DVfLJ8RQKO5PYmjw4KVpA", "tool_input": "{\"task\": \"We need to analyze 
Amazon product sales data. Please perform the following tasks:\\n\\n1. Create or download a sample Amazon product 
sales dataset suitable for analysis\\n2. Load the dataset\\n3. Examine basic properties (shape, info, 
describe)\\n4. Check for missing values and outliers\\n5. Perform basic statistical analysis (mean, median, 
variance)\\n6. Display a summary of your findings\\n\\nPlease provide code with explanations and output the initial
data exploration results in a clear, readable format.\"}"}

KeyboardInterrupt: 

In [33]:
agent_arn = launch_result.agent_arn
agent_arn

'arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/runtime-GxV7G995xb'

In [8]:
import json
import boto3
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

def parse_sse_data(sse_bytes):
    if not sse_bytes or len(sse_bytes) == 0:
        return None

    try:
        text = sse_bytes.decode('utf-8').strip()
        if not text or text == '':
            return None

        if text.startswith('data: '):
            json_text = text[6:].strip()
            if json_text:
                return json.loads(json_text)
        else:
            # 순수 JSON인 경우
            return json.loads(text)

    except Exception as e:
        # 디버깅용으로만 출력
        # print(f"Parse error: {e}")
        pass

    return None
      

# agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)
agent_arn = "arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/runtime-GxV7G995xb"



boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    #payload=json.dumps({"prompt": "What is the weather now?"})
    payload=json.dumps({"prompt": '이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요. 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.'})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for event in boto3_response["response"].iter_lines(chunk_size=1):
        event = parse_sse_data(event)
        if event is None:  # None 체크 추가
            continue
        #print ("3", type(event), event)
        if event.get("event_type") == "text_chunk":
            # Print streaming text chunks in real-time
            print(event.get("data", ""), end="", flush=True)
            #print (event)
        elif event.get("event_type") == "reasoning":
            # Print reasoning tokens in real-time (can be used separately later)
            print(event.get("reasoning_text", ""), end="", flush=True)
            #print (event)
        
        ## 툴 프린팅은 수정해야 함!! 스트리밍 될 수 있는 지 확인하기 event를 출력하면 어떻게 넘어 오는지 확인 가능함. 
        elif event.get("event_type") == "tool_use": 
            # Print tool usage events
            tool_name = event.get("tool_name", "unknown")
            tool_input = event.get("tool_input", "")
            #print (event)
            
            # Try to parse tool_input as JSON and extract task
            try:
                import json
                tool_data = json.loads(tool_input)
                task = tool_data.get("task", tool_input)
                #print(f"\n[TOOL] Using {tool_name}", flush=True)
                print(f"[TOOL] Task: {task}{'...' if len(task) > 100 else ''}", flush=True)
            except:
                pass
                #print(f"\n[TOOL] Using {tool_name}...", flush=True)
        elif event.get("type") == "final_result":
            print(f"\n\n[FINAL] Agent: {event.get('agent')}")
            print(f"[FINAL] Response: {event.get('response')}")
        else:
            # Print other events
            print(f"\n[EVENT] {event}")
    #     if line:
    #         line = line.decode("utf-8")
    #         if line.startswith("data: "):
    #             line = line[6:]
    #             print ("11", line)
    #             #logger.info(line)
    #             content.append(line)
    # display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            print ("6", event)
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [28]:
import boto3
import json
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    #payload=json.dumps({"prompt": "What is the weather now?"})
    payload=json.dumps({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
    #agentcore_runtime.invoke({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
)
# if "text/event-stream" in boto3_response.get("contentType", ""):
#     content = []
#     for line in boto3_response["response"].iter_lines(chunk_size=1):
#         if line:
#             line = line.decode("utf-8")
#             if line.startswith("data: "):
#                 line = line[6:]
#                 logger.info(line)
#                 content.append(line)
#     display(Markdown("\n".join(content)))
# else:
#     try:
#         events = []
#         for event in boto3_response.get("response", []):
#             events.append(event)
#     except Exception as e:
#         events = [f"Error reading EventStream: {e}"]
#     display(Markdown(json.loads(events[0].decode("utf-8"))))

In [29]:
boto3_response

{'ResponseMetadata': {'RequestId': 'd2f783a1-80c5-4728-ac84-a1ed339d173f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 06 Aug 2025 02:03:21 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd2f783a1-80c5-4728-ac84-a1ed339d173f',
   'baggage': 'Self=1-6892b7d7-4c06c892527a66162693c8a4,session.id=ac7fb602-2130-490d-8ca2-aa86de3be3b3',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'ac7fb602-2130-490d-8ca2-aa86de3be3b3',
   'x-amzn-trace-id': 'Root=1-6892b7d7-4705acbd445e51fd08765686;Self=1-6892b7d7-4c06c892527a66162693c8a4'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'ac7fb602-2130-490d-8ca2-aa86de3be3b3',
 'traceId': 'Root=1-6892b7d7-4705acbd445e51fd08765686;Self=1-6892b7d7-4c06c892527a66162693c8a4',
 'baggage': 'Self=1-6892b7d7-4c06c892527a66162693c8a4,session.id=ac7fb602-2130-490d-8ca2-aa86de3be3b3',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': <botocore.respo

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!